# Training and Modelling for the TS based Cov-Matrix Picker

In [30]:
import torch, os
import pandas as pd
import numpy as np
import pickle
from math import factorial as fac
import matplotlib
import random as rd
from random import sample 
import matplotlib.pyplot as plt 
from itertools import combinations as combs
from numpy import random as npr
import torch.nn as nn
#import torch.distributed as dist
#from torch.nn.parallel import DistributedDataParallel

## Support Functions

In [49]:
def nChoosek(n,k):
    return fac(n) // fac(k) // fac(n-k)

def which_mat(TTS, choice, n = [5, 5*3, 5*5, 5*9, 5*13], p = [0], channel_use = 1):
    """
    Funciton which can be easilly calle din order to implament which_mat as part of a larger function
    """
    return get_mat(TTS, n[choice[0]], p[0], channel_use)

def get_mat(TTS, n,p = 0, channel_use = 1):
    """
    Funciton that takes in 
    TTS Tesnor of Time Serieses, and given 
    n number of period to use and
    p  last index period to use, 
    return a covariance matrix of the data for that period in terms of the values in chanel channel_use
    """
    
    d1, d2 = TTS[0], TTS[1]
    #insert function to partition TTS correctly
    
    #creating cova matrix estimation desired
    d = [d1, d1]
    return pd.DataFrame({"S%d" %i: d for i,d in  enumerate(d)}).cov()


def cov_matrix_loss(A,B, type_use = 1):
    """
    Get distance between matracies based either on component wise distance or using eigenvalues
    """
    if type_use == 0:
        ind = np.triu_indices(2)
        loss = nn.MSELoss()
        return loss(A[ind], B[ind]) 
    
    elif type_use == 1:
        return torch.sqrt(torch.sum(torch.pow(torch.log(torch.eig(A-B))),2))
    

## NN Models

In [15]:
class TS_based_simple(nn.Module):
    def __init__(self, num_outs,
                 kernel = 5, in_channels = 4,  hidden_size = 128):
        """
        
        
        """
        super(TS_based_simple, self).__init__()
        
        pads = int(kernel // 2)
        
        self.downconv1 = nn.Sequential(
            nn.Conv1d(in_channels = in_channels, out_channels = hidden_size, 
                      stride = 2,kernel_size = kernel, padding = pads ),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.MaxPool1d(2)
            )
        self.downconv2 = nn.Sequential(
            nn.Conv1d(in_channels = hidden_size, out_channels = int(hidden_size/2), 
                      kernel_size = int(kernel-2), stride = 2, padding = int(pads-1) ),
            nn.BatchNorm1d(int(hidden_size/2)),
            nn.ReLU(),
            nn.MaxPool1d(2)
            )
        
        self.downconv3 = nn.Sequential(
            nn.Conv1d(in_channels = int(hidden_size/2), out_channels = int(hidden_size/(2**2)), 
                      kernel_size = int(kernel-2), stride = 2, padding = (pads-1) ),
            nn.BatchNorm1d(int(hidden_size/(2**2))),
            nn.ReLU(),
            nn.MaxPool1d(2)
            )
        
        self.downconvLast = nn.Sequential(
            nn.Conv1d(in_channels = int(hidden_size/(2**2)) , out_channels = num_outs, 
                      kernel_size = int(kernel-2), padding = int(pads-1)),
            nn.BatchNorm1d(num_outs),
            nn.ReLU(),
            nn.MaxPool1d(2)
            )
        
        self.picker = nn.Linear(in_features = num_outs, out_features = num_outs)
        
    
    def forward(self, input):
        
        input_1, input_2 = input[0], input[1]
        
        #run the model on the first TS
        self.c11 = self.cov1(input_1)
        self.c12 = self.cov2(self.c11)
        self.c13 = self.cov3(self.c12)
        
        #run the model on the second TS
        self.c21 = self.cov1(input_2)
        self.c22 = self.cov2(self.c11)
        self.c23 = self.cov3(self.c12)
        
        #concat the processed data and downconvthat
        self.cl = self.downconvLast(torch.cat((self.c23, self.c3), dim=1))
        
        #concat TS based data to data representative of other info
        self.last = self.picker(self.cl)
        

class TS_based_wfactors(nn.Module):
    def __init__(self, num_outs, kernel = 5, channels_in =4, hidden_size = 128):
        """
        
        
        """
        super(TS_based_wfactors, self).__init__()
        
        pads  = int(kernel//2)
        
        self.downconv1 = nn.Sequential(
            nn.Conv1d(in_channels = channels_in, out_channels = hidden_size, 
                      stride = 2,kernel_size = kernel, padding = pads ),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.MaxPool1d(2)
            )
        self.downconv2 = nn.Sequential(
            nn.Conv1d(in_channels = hidden_size, out_channels = int(hidden_size/2), 
                      kernel_size = int(kernel-2), stride = 2, padding = int(pads-1) ),
            nn.BatchNorm1d(int(hidden_size/2)),
            nn.ReLU(),
            nn.MaxPool1d(2)
            )
        
        self.downconv3 = nn.Sequential(
            nn.Conv1d(in_channels = int(hidden_size/2), out_channels = int(hidden_size/(2**2)), 
                      kernel_size = int(kernel-2), stride = 2, padding = int(pads-1)),
            nn.BatchNorm1d(int(hidden_size/(2**2))),
            nn.ReLU(),
            nn.MaxPool1d(2)
            )
        
        self.downconvLast = nn.Sequential(
            nn.Conv1d(in_channels = int(hidden_size/(2**2)) , out_channels = num_outs, 
                      kernel_size = int(kernel-2), padding = int(pads-1)),
            nn.BatchNorm1d(num_outs),
            nn.ReLU(),
            nn.MaxPool1d(2)
            )
        
        self.picker = nn.Linear(in_channels = num_outs , out_channels = num_outs)
        
    
    def forward(self, input):
        
        input_1, input_2 = input[0], input[1]
        
        #run the model on the first TS
        self.c11 = self.cov1(input_1)
        self.c12 = self.cov2(self.c11)
        self.c13 = self.cov3(self.c12)
        
        #run the model on the second TS
        self.c21 = self.cov1(input_2)
        self.c22 = self.cov2(self.c11)
        self.c23 = self.cov3(self.c12)
        
        #concat the processed data and downconvthat
        self.cl = self.downconvLast(torch.cat((self.c23, self.c3), dim=1))
        
        #concat TS based data to data representative of other info
        self.last = self.picker(self.cl)
        

## Trainer

In [82]:
def asset_combinations(x,y):
    t = [0,0,0,1]
    """
    Notice:
        if x = SP400 then t[0] or t[1] = 1
        if y = SP500 then t[2] or t[1] = 1
    
    """
    if x == y:
        t[3] = 0
        if x == "SP400":
            t[0] = 1
        else:
            t[2] = 1
    elif x == "SP400":
        t[3] = 0
        t[1] = 1
    return t
    
def create_data_sets(list_periods, list_combs, data_use):
    type_1 = data_use[0].keys()
    
    #which indecies does each asset bellong to 
    which_type = lambda x: "SP400" if x in type_1 else "SP500"
    met_dat = [[asset_combinations(which_type(a[0]), which_type(a[1]))] for a in list_combs]
    which_type = lambda x: 0 if x in type_1 else 1
    name_dat = [[which_type(a[0]), which_type(a[1])] for a in list_combs]
    dater_test = []
    dater_train = []
    """
    first_is_first = lambda x: 0 if (x[0] == 1 or x[1] == 1) else 1
    second_is_second = lambda y: 1 if (y[2] == 1 or y[1] == 1) else 0
    
    print(list_combs[0])
    print(met_dat[0])
    print(len(data_use[first_is_first(met_dat[1][0])][list_combs[1][0]]))
    print(data_use[first_is_first(met_dat[1][0])][list_combs[1][0]][1302])
    print(list_periods[0])
    print(len(list_combs))
    print(len(met_dat))"""

   
    checker_n = lambda x,y,j :  x <= min(len(y[name_dat[j][0]][list_combs[j][0]]), 
                                         len(y[name_dat[j][1]][list_combs[j][1]]))
    
    k = 0
    for i in range(len(list_combs)):
         #test data set
        for period_good_good in list(range(list_periods[i][0][0], list_periods[i][0][1] - 1)):
            if checker_n(period_good_good,data_use,i):
                try:
                    dater_test.append([[data_use[name_dat[i][0]][list_combs[i][0]][period_good_good],
                               data_use[name_dat[i][1]][list_combs[i][1]][period_good_good]], 
                                       met_dat[i][0]])
                except:
                    k += 1
            #train data set
        k_list = list(range(0,list_periods[i][1][0])) + list(range(list_periods[i][1][1], list_periods[i][1][2]))
        for period_good_good in k_list :
            if checker_n(period_good_good,data_use,i):
                try: 
                    dater_train.append([[data_use[name_dat[i][0]][list_combs[i][0]][period_good_good],
                       data_use[name_dat[i][1]][list_combs[i][1]][period_good_good]],
                       met_dat[i][0]])
                except:
                    k += 1
    print(str(k/(len(list_periods)*list_periods[0][1][2])) + "% of dates were skipped")
                                    
    
    return dater_train, dater_test

def test_dates(bot_t, p_to_use):
    return list(range(bot_t, p_to_use ))

def train_dates(bot_tr, top_tr, max_p):
    return list(range(bot_tr)) + list(range(top_tr, max_p))

def pick_a_date(max_period, args_use):
    """
    Giving max final period, return a breakdown of periods between training and testing
    """
    ratio = args_use.ratio
    #boundary values
    validation_samples = min(max_period*(100-ratio), args_use.periods_testing) 
    
    #test periods
    p_use = rd.randint(validation_samples,max_period)
    botim_test = p_use - validation_samples
    
    #train periods
    top_train = p_use + args_use.periods_approximate
    bottom_train = botim_test - args_use.periods_approximate
    
    return [[botim_test, p_use], [bottom_train, top_train, max_period]]

def train_set(data_use, args):
    
    #how many periods 
    num_periods = list(data_use[0].values())[0].size()[0]
    
    #picking which asset combinations to use
    num_combs = min(args.max_train, nChoosek(args.num_stocks_each,2))
    all_keys = list(data_use[0].keys()) + list(data_use[1].keys())
    which_combs = sample(list(combs(all_keys,2)), num_combs)
    
    
    #how do we pick which periods to test on
    if args.testing_dates == "Random":
        #get a list of differnt periods for Testing and same dates for Trainig for all cases
        p_use = [pick_a_date(num_periods, args) for i in range(num_combs)]
    
    else:
        #get a list of the same periods for Testing and same dates for Trainig for all cases
        p_use = np.repeat(pick_a_date(num_periods, args), num_combs)
        
    
    #give define data sets
    
    
    
    return create_data_sets(p_use, which_combs, data_use)



In [73]:
def train(train_d, args, mod_use = None):
    #setting up the packages
    torch.set_num_threads(5)
    npr.seed(args.seed)
    dir_save_to = "results/" + args.dir_name
    if not os.path.exists(dir_save_to):
        os.makedirs(dir_save_to)
    
    
    #type of metric to use for loss:
    l_t = 1
    if arg.loss_type == "L2":
        l_t = 0
    
    
    #which model to implament
    if mod_use is None:
        if args.model_use == "Mod1":
              mod_use = TS_based_simple(num_outs = args.k_size) #args.num_filters, num_colours, arg.num_in_chans)
        elif args.model_use == "Mod1wFactors":
              mod_use = TS_based__wfactors(num_outs = args.k_size) #, args.num_filters, num_colours, arg.num_in_chans)
        
    #setting up the model's optimizaer
    optimizer = torch.optim.Adam(mod_use.parameters(), lr=args.lrn_rate)
    
    #where we are going to gather data
    hist_tr_loss = []
    hist_tt_loss = []
    
    #let's start training
    for epoch in range(arg.num_epochs):
        epoch_loss = []
        
        #let's train
        training_d, validation_d = train_set(train_d, args)
        for input_d, output_d in training_d:
            
            #setup optimizer
            optimizer.zero_grad()
            
            #set up optimization
            output_choice = mod_use(input_d)
            CM_use = which_mat(input_d, output_choice)
            lossing_me = cov_matrix_loss(output_d, CM_use, l_t)
            
            #optimizing
            lossing_me.backward()
            optimizer.step()
            
            #adding performance
            epoch_loss.append(lossing.data.item())
        
        #printing performance on epoch
        mean_loss = np.mean(epoch_loss)
        hist_tr_loss.append(mean_loss)
        print('Epoch [%d/%d], T Loss: %.4f' % (epoch+1, args.num_epochs, mean_loss))
        
        
        
        #let's validate
        temp_validation = []
        for input_d, output_d in validation_d:
            #setup optimizer
            optimizer.zero_grad()
            
            #set up optimization
            output_choice = mod_use(input_d)
            CM_use = which_mat(input_d, output_choice)
            lossing_me = cov_matrix_loss(output_d, CM_use, l_t)
            
            
            #adding performance
            temp_validation.append(lossing.data.item())
        
        hist_tt_loss = []
        #printing performance on epoch
        mean_loss = np.mean(temp_validation)
        hist_tt_loss.append(mean_loss)
        print('Epoch [%d/%d], V Loss: %.4f' % (epoch+1, args.num_epochs, mean_loss))
    
    
    plt.figure()
    plt.plot(hist_tt_loss, "ro-", label="Train")
    plt.plot(hist_tr_loss, "go-", label="Validation")
    plt.legend()
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.savefig(save_dir+"/training_curve.png")

    if args.checkpoint:
        print('Saving model...')
        torch.save(mod_use.state_dict(), args.save_model_as)
    
    return mod_use
        